In [1]:
import pandas as pd
from modules.data import Data
# from modules.search import Search
# from modules.video import Video
# from modules.analyze import Analyze
# from tqdm.notebook import tqdm
import csv

In [2]:
df_comments = pd.read_csv("generated_tables/comment_threads.csv", index_col=False)
df_video = pd.read_csv("generated_tables/video_info.csv", index_col=False)
df_video_tagged = pd.read_csv("tagged_v3.csv", index_col=False)

In [3]:
data_obj = Data()

In [4]:
df_video_numbered = data_obj.generate_id_lookup(df=df_video, unique_column="video_url", len_identifier=5, rename_to="video_number")
df_comments["video_url"] = df_comments["video_ID"].apply(lambda video_ID: data_obj.generate_url("video", video_ID))
print("Generated video_url")
df_comments = pd.merge(left=df_comments, right=df_video_numbered, on="video_url", validate="many_to_one", suffixes=("", "_DROP")).filter(regex="^(?!.*_DROP)")
print("Merged")
df_comment_thread_numbered = data_obj.generate_id_lookup(df=df_comments, unique_column="comment_thread_ID", len_identifier=6, rename_to="comment_thread_number")
print("Generated comment_thread_number")
df_comments = pd.merge(left=df_comments, right=df_comment_thread_numbered, on="comment_thread_ID", validate="many_to_one", suffixes=("", "_DROP")).filter(regex="^(?!.*_DROP)")
print("Merged")
df_comment_numbered = data_obj.generate_id_lookup(df=df_comments, unique_column="comment_ID", len_identifier=7, rename_to="comment_number")
print("Generated comment_number")
df_comments = pd.merge(left=df_comments, right=df_comment_numbered, on="comment_ID", validate="many_to_one", suffixes=("", "_DROP")).filter(regex="^(?!.*_DROP)")
print("Merged")
df_author_numbered = data_obj.generate_id_lookup(df=df_comments, unique_column="author_channel_ID", len_identifier=7, rename_to="author_number")
print("Generated author_number")
df_comments = pd.merge(left=df_comments, right=df_author_numbered, on="author_channel_ID", validate="many_to_one", suffixes=("", "_DROP")).filter(regex="^(?!.*_DROP)")
print("Merged")

Generated video_url
Merged
Generated comment_thread_number
Merged
Generated comment_number
Merged
Generated author_number
Merged


In [5]:
df_comments = df_comments.drop(columns=["video_ID", "comment_thread_ID", "comment_ID", "author_channel_ID", "video_url"])

In [6]:
df_comments.head()

,comment,like_count,updated_at,publish_time,video_number,comment_thread_number,comment_number,author_number
0,What is the 5 dollor one called,0.0,2021-06-24T18:33:38Z,2021-06-24T18:33:38Z,00001,000000,0000000,0000000
1,Can you do a review on the lava 2 vape,0.0,2021-06-21T06:36:22Z,2021-06-21T06:36:22Z,00001,000001,0000001,0000001
2,When you’re feining so bad u gotta watch other...,9.0,2021-06-21T05:20:29Z,2021-06-21T05:20:29Z,00001,000002,0000002,0000002
3,i fr thought it was just me 🌚,0.0,2021-06-23T07:31:18Z,2021-06-23T07:31:18Z,00001,000002,0000003,0000003
4,I recharged and filled any cheap disposable th...,0.0,2021-06-17T04:16:27Z,2021-06-17T04:16:27Z,00001,000004,0000004,0000004


In [7]:
# Anonymize the channel URLs
df_channel_numbered = data_obj.generate_id_lookup(df=df_video, unique_column="channel_url", len_identifier=7, rename_to="channel_number")
df_video = pd.merge(left=df_video, right=df_channel_numbered, on="channel_url", validate="many_to_one", suffixes=("", "_DROP")).filter(regex="^(?!.*_DROP)")
df_video = df_video.drop(columns=["channel_url"])
# Generate the anonymized video dataset
df_video = pd.merge(left=df_video, right=df_video_numbered, on="video_url", validate="one_to_one", suffixes=("", "_DROP")).filter(regex="^(?!.*_DROP)")
# Merge with the tagged dataset
df_video = pd.merge(left=df_video, right=df_video_tagged, on="video_url", validate="many_to_one", suffixes=("", "_DROP")).filter(regex="^(?!.*_DROP)")
# Drop video_url
df_video = df_video.drop(columns=["video_url"])

In [8]:
# Save the mapping
df_video_numbered.to_csv("unique_id_map/video_numbered.csv", index=False, quoting=csv.QUOTE_ALL)
df_comment_thread_numbered.to_csv("unique_id_map/comment_thread_numbered.csv", index=False, quoting=csv.QUOTE_ALL)
df_comment_numbered.to_csv("unique_id_map/comment_numbered.csv", index=False, quoting=csv.QUOTE_ALL)
df_author_numbered.to_csv("unique_id_map/author_numbered.csv", index=False, quoting=csv.QUOTE_ALL)
df_channel_numbered.to_csv("unique_id_map/channel_numbered.csv", index=False, quoting=csv.QUOTE_ALL)

In [9]:
# Save the anonymized comments dataset
df_comments.to_csv("unique_id_map/comments_anonymized.csv", index=False, quoting=csv.QUOTE_ALL)

In [10]:
df_video.head()

,video_title,video_description,view_count,like_count,dislike_count,favorite_count,comment_count,channel_url,publish_time,video_duration,channel_number,video_number,category,theme
0,#vAustinL Vape Trick Tutorial - How to Vape O...,"If you’re new to tricking, have no fear. It ju...",37391,565,18,0,23,https://youtube.com/channel/UCO9ugWO6L7Rfk7Uxi...,2019-05-08T06:37:58Z,PT5M11S,0000000,00000,Risky,Vaping tricks
1,$100 Vape vs $5 Vape,Expensive box mod vs a regular disposable stic...,363589,4730,237,0,217,https://youtube.com/channel/UCfmwCI6xnGWRw2wjk...,2019-09-21T17:19:33Z,PT5M25S,0000001,00001,Risky,Marketing
2,New Studio Vape Trick Session,Catch me live on twitch! https://www.twitch.tv...,2683795,64849,1075,0,1881,https://youtube.com/channel/UCZ4p4AjF8kmxZOW5Z...,2019-01-11T20:16:37Z,PT3M6S,0000002,00002,Risky,Vaping tricks
3,Vape Trick Tutorial - How to: Bane French Inhale,Catch me live on twitch! https://www.twitch.tv...,4706152,68394,1511,0,1647,https://youtube.com/channel/UCZ4p4AjF8kmxZOW5Z...,2017-04-14T22:58:04Z,PT3M24S,0000002,00031,Risky,Vaping tricks
4,Amazing Vape Bubble Compilation,Catch me live on twitch! https://www.twitch.tv...,17299923,242855,7676,0,7715,https://youtube.com/channel/UCZ4p4AjF8kmxZOW5Z...,2018-05-26T19:42:43Z,PT3M16S,0000002,00042,Risky,Vaping tricks


In [11]:
# Save the dataset
df_video.to_csv("unique_id_map/videos_anonymized.csv", index=False, quoting=csv.QUOTE_ALL)